In [157]:
#Needs to use selenium. 
import sys
#Below you must set path to where webdriver is installed. This path will be different for you.
sys.path.insert(0,'Z:\\jrg363\\Selenium Python\\chromedriver\\chromedriver-win64')

#Make sure to install all packages.
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


import re
import pandas as pd
import time

In [158]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [159]:
#Start web driver
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)

In [160]:
#Import text files
import os
os.chdir("Z:\\jrg363\\workshops SP25\\Guest Lecture Apr25\\API filing_details")
data = pd.read_excel("api_details.xlsx")


In [161]:
ciks_sample = data['cik_Number'][0:2]

In [162]:
ciks_sample

0     700764
1    1819438
Name: cik_Number, dtype: int64

In [163]:
#Go to chatgpt
#https://chatgpt.com/
driver.get("https://deepai.org/chat")
time.sleep(5)


In [164]:
def get_text_after_substrings(text, substrings):
    for substring in substrings:
        index = text.find(substring)
        if index != -1:
            text = text[index-len(substring) + len(substring):]
    return text

In [165]:
#Counter for going through text files
count = 0
#Empty list that will store responses from copilot. 
AIResp = []

#Loop through cik numbers that created text file for. 
for cik in ciks_sample:
  #File name for importing text file
    filenamei = str(cik) + ".txt"
    #New file name that contains copilot summary. 
    filenamee = str(cik) + "researchAndDevelopmentSummary" + ".txt"
    #Import txt file.
    f = open(filenamei, 'r')
    #Read text
    content = f.read()
    #Close text file
    f.close()
    #Search for these strings for start of section
    substrings = ["Research and Development", "Research &#38; Development"]
    result = get_text_after_substrings(content, substrings) 
    #Replace &#160; with nothing
    result = result.replace("&#160;", "")
    
    prompt = "summarize research and development secction of following text: "
    prompt_enter = prompt + result
    prompt_enter = prompt_enter[0:2000]
    prompt_enter = prompt_enter.replace("\n", "")
    #Enter into prompt
    keywords = driver.find_element(By.XPATH, "//textarea[@id='persistentChatbox']")
    keywords.send_keys(prompt_enter)

    submit = driver.find_element(By.XPATH, "//button[@id='chatSubmitButton']")
    submit.click()
    time.sleep(20)
    response = driver.find_elements(By.XPATH, "//div[@class='outputBox']")
    AIResp.append(response[count].text)
    count = count + 1

In [166]:
#Then combine into excel spreadsheet and export. 
cik_df = pd.DataFrame({'ciks': ciks_sample, 'API Response': AIResp})

In [167]:
display(cik_df)

,ciks,API Response
0,700764,The research and development section highlight...
1,1819438,The research and development section emphasize...


In [168]:
cik_df.to_excel("Summary of research and development.xlsx")